# Overview
The script reads an mtx file called sample_4.mtx in the sample_dataset folder.  
It then displays it in a cytoscape graph.  
Requires the ipysytoscape library and Python 3  
Tested on jupyter notebook with version pandas version 1.3.4

In [1]:
!pip install ipycytoscape

In [2]:
!pip install pandas

In [3]:
# Import all important libraries
from scipy.io import mmread
from scipy.io import mminfo
import pandas as pd
import numpy as np
import requests
# For reading the file
import os
import sys

# For transforming into cytoscape
import ipycytoscape
import json
import ipywidgets

# Double checking versions
print(pd.__version__) #had 1.3.4
print(ipycytoscape.__version__) #had 1.3.2
print(json.__version__) #had 2.0.9
print(ipywidgets.__version__) #had 7.6.5

2.2.2
1.3.3
2.0.9
8.1.3


In [4]:
# Set path and open file for reading
# filespec = os.fspath('../sample_dataset/sample_4.mtx')
# stream = open(filespec, 'r')

In [5]:
# #line = stream.readline()
# # Skip lines that start with %
# while line and line[0] in ['%', 37]:
#     line = stream.readline()
# # Skip empty lines
# while not line.strip():
#     line = stream.readline()

In [6]:
import requests
import pandas as pd
import json

# Define the API URL
URL = 'https://api.opendata.ocs.oraclecloud.com/data/landsat-4-5-tm/'


In [7]:
def fetch_json_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

data = fetch_json_data(URL)


In [8]:
def print_json_data(data):
    print("JSON Data:", json.dumps(data, indent=2))

if data:
    print_json_data(data)


JSON Data: {
  "objects": [],
  "prefixes": [
    "001/"
  ],
  "nextStartWith": "001/058/LT05_L1TP_001058_20110430_20200822_02_T1_VAA.TIF"
}


In [9]:
def extract_relevant_parts(data):
    objects = data.get('objects', [])
    prefixes = data.get('prefixes', [])
    next_start_with = data.get('nextStartWith', '')
    return objects, prefixes, next_start_with

objects, prefixes, next_start_with = extract_relevant_parts(data)

def print_extracted_parts(objects, prefixes, next_start_with):
    print("Objects:", objects)
    print("Prefixes:", prefixes)
    print("Next Start With:", next_start_with)

print_extracted_parts(objects, prefixes, next_start_with)


Objects: []
Prefixes: ['001/']
Next Start With: 001/058/LT05_L1TP_001058_20110430_20200822_02_T1_VAA.TIF


In [10]:
def initialize_data_structures():
    return [], [], 1  # edge_array, node_array, counter

edge_array, node_array, counter = initialize_data_structures()


In [11]:
def process_line(line, edge_array, node_array, counter):
    if line and line[0] not in ['%', '37']:
        split_line = line.split('/')
        print("Split line:", split_line)
        
        numeric_parts = [part for part in split_line if part.isdigit()]
        if len(numeric_parts) >= 2:
            try:
                rows, cols = map(int, numeric_parts[:2])
                entries = 1  # Default entries to 1 if not specified
                
                print("rows: ", rows, " cols: ", cols, " entries: ", entries)
                
                node_source = {'id': rows, 'label': rows, 'classes': rows}
                node_target = {'id': cols, 'label': cols, 'classes': cols}
                if node_source not in node_array:
                    node_array.append(node_source)
                if node_target not in node_array:
                    node_array.append(node_target)
                
                edge_entry = {'id': 'line ' + str(counter), 'source': rows, 'target': cols, 'weight': entries}
                edge_array.append(edge_entry)
                
                counter += 1
            except ValueError:
                print(f"Skipping invalid line due to ValueError: {line}")
        else:
            print(f"Skipping invalid line: {line}")
    return edge_array, node_array, counter

edge_array, node_array, counter = process_line(next_start_with, edge_array, node_array, counter)


Split line: ['001', '058', 'LT05_L1TP_001058_20110430_20200822_02_T1_VAA.TIF']
rows:  1  cols:  58  entries:  1


In [12]:
def convert_to_dataframe(edge_array, node_array):
    node_df = pd.DataFrame(node_array)
    edge_df = pd.DataFrame(edge_array, columns=['id', 'source', 'target', 'weight'])
    return node_df, edge_df

node_df, edge_df = convert_to_dataframe(edge_array, node_array)

def print_dataframes(node_df, edge_df):
    print("Node DataFrame Columns:", node_df.columns)
    print("Node DataFrame:")
    print(node_df)
    print("\nEdge DataFrame:")
    print(edge_df)

print_dataframes(node_df, edge_df)


Node DataFrame Columns: Index(['id', 'label', 'classes'], dtype='object')
Node DataFrame:
   id  label  classes
0   1      1        1
1  58     58       58

Edge DataFrame:
       id  source  target  weight
0  line 1       1      58       1


In [13]:
def assign_node_colors(node_df):
    if not node_df.empty:
        node_df['background-color'] = 'red'
        node_df.loc[node_df['id'] <= 40, 'background-color'] = 'blue'
        node_df.loc[node_df['id'] >= 100, 'background-color'] = 'orange'
    return node_df

node_df = assign_node_colors(node_df)


In [14]:
def main():
    data = fetch_json_data(URL)
    if data:
        print_json_data(data)
        objects, prefixes, next_start_with = extract_relevant_parts(data)
        print_extracted_parts(objects, prefixes, next_start_with)
        edge_array, node_array, counter = initialize_data_structures()
        edge_array, node_array, counter = process_line(next_start_with, edge_array, node_array, counter)
        node_df, edge_df = convert_to_dataframe(edge_array, node_array)
        node_df = assign_node_colors(node_df)
        print_dataframes(node_df, edge_df)

main()


JSON Data: {
  "objects": [],
  "prefixes": [
    "001/"
  ],
  "nextStartWith": "001/058/LT05_L1TP_001058_20110430_20200822_02_T1_VAA.TIF"
}
Objects: []
Prefixes: ['001/']
Next Start With: 001/058/LT05_L1TP_001058_20110430_20200822_02_T1_VAA.TIF
Split line: ['001', '058', 'LT05_L1TP_001058_20110430_20200822_02_T1_VAA.TIF']
rows:  1  cols:  58  entries:  1
Node DataFrame Columns: Index(['id', 'label', 'classes', 'background-color'], dtype='object')
Node DataFrame:
   id  label  classes background-color
0   1      1        1             blue
1  58     58       58              red

Edge DataFrame:
       id  source  target  weight
0  line 1       1      58       1


In [15]:
def convert_to_dict_records(nodes_df, edges_df):
    nodes_dict = nodes_df.to_dict('records')
    edges_dict = edges_df.to_dict('records')
    return nodes_dict, edges_dict


In [16]:
def build_nodes_graph_list(nodes_dict):
    data_keys = ['id', 'label', 'classes']
    position_keys = ['position_x', 'position_y']
    rest_keys = ['score', 'idInt', 'name', 'group', 'removed', 'selected', 'selectable', 'locked', 'grabbed', 'grabbable']

    nodes_graph_list = []
    for node in nodes_dict:
        data_sub_dict = {'data': {el: node[el] for el in data_keys}}
        rest_sub_dict = {el: node[el] for el in node.keys() if el in rest_keys}
        posi_sub_dict = {}
        if 'position_x' in node.keys() and 'position_y' in node.keys():
            posi_sub_dict = {'position': {el: node[el] for el in position_keys}}

        dict_node = {**data_sub_dict, **rest_sub_dict, **posi_sub_dict}
        nodes_graph_list.append(dict_node)

    return nodes_graph_list


In [17]:
def build_edges_graph_list(edges_dict):
    data_keys = ['id', 'source', 'target']
    data_keys2 = ['label', 'classes']
    rest_keys = ['score', 'weight', 'group', 'networkId', 'networkGroupId', 'intn', 'rIntnId', 'removed', 'selected', 'selectable', 'locked', 'grabbed', 'grabbable']

    edges_graph_list = []
    for edge in edges_dict:
        data_sub_dict = {el: edge[el] for el in data_keys}
        data_sub_dict2 = {el: edge[el] for el in edge.keys() if el in data_keys2}
        rest_sub_dict = {el: edge[el] for el in edge.keys() if el in rest_keys}

        dict_edge = {'data': {**data_sub_dict, **data_sub_dict2}, **rest_sub_dict}
        edges_graph_list.append(dict_edge)

    return edges_graph_list


In [18]:
def build_style_elements_list(nodes_dict, edges_dict):
    all_node_style = ['background-color', 'background-opacity', 'font-family', 'font-size', 'label', 'width', 'shape', 'height', 'text-valign', 'text-halign']
    all_edge_style = ['background-color', 'background-opacity', 'font-family', 'font-size', 'label', 'width', 'line-color']

    style_elements = []
    for node in nodes_dict:
        node_dict = {'selector': f'node[id = "{node["id"]}"]'}
        style_dict = {"style": {el: node[el] for el in node.keys() if el in all_node_style}}
        node_dict.update(style_dict)
        style_elements.append(node_dict)

    for edge in edges_dict:
        edge_dict = {'selector': f'edge[id = "{edge["id"]}"]'}
        style_dict = {"style": {el: edge[el] for el in edge.keys() if el in all_edge_style}}
        edge_dict.update(style_dict)
        style_elements.append(edge_dict)

    return style_elements


In [19]:
import ipycytoscape

def transform_into_ipycytoscape(nodes_df, edges_df):
    nodes_dict, edges_dict = convert_to_dict_records(nodes_df, edges_df)
    
    nodes_graph_list = build_nodes_graph_list(nodes_dict)
    edges_graph_list = build_edges_graph_list(edges_dict)
    
    total_graph_dict = {'nodes': nodes_graph_list, 'edges': edges_graph_list}

    style_elements = build_style_elements_list(nodes_dict, edges_dict)

    data_graph = json.dumps(total_graph_dict)
    json_to_python = json.loads(data_graph)
    
    result_cyto = ipycytoscape.CytoscapeWidget()
    result_cyto.graph.add_graph_from_json(json_to_python)
    result_cyto.set_style(style_elements)
    
    return result_cyto


In [20]:
G = transform_into_ipycytoscape(node_df, edge_df)
display(G)


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node[id = "1"]', 'style': {'…